# Part IX — Deployment and Production Operations  
## 37. Deployment Options (Concepts + Practical Paths)

This chapter is a deployment “playbook.” You’ll learn the industry-standard ways
Django is deployed, how to choose between them, and how to wire **everything** you
already built into a production system:

- Django web (WSGI or ASGI)
- static files (collectstatic)
- media uploads (persistent storage)
- PostgreSQL
- Redis (cache / Channels / Celery broker)
- Celery workers + beat
- domain + HTTPS + reverse proxy (Nginx)
- environment variables + secrets
- migrations and rollback discipline
- health/readiness endpoints
- logs/monitoring hooks

You will finish with at least one complete deployment path you can repeat.

---

## 37.0 Learning Outcomes

By the end you should be able to:

1. Choose a deployment approach:
   - VPS + Nginx + Gunicorn/Uvicorn (+ systemd)
   - PaaS (Render/Heroku-like)
   - Containers (Docker) on a VM or container platform
2. Correctly deploy:
   - WSGI (Gunicorn) for classic Django
   - ASGI (Uvicorn) for async + Channels/WebSockets
3. Run and manage separate processes:
   - web
   - celery worker
   - celery beat
4. Handle static/media correctly:
   - `collectstatic`
   - persistent media (S3 or disk volume)
5. Apply production settings and security:
   - DEBUG off
   - secret key in env
   - allowed hosts / CSRF trusted origins
   - secure cookies / HTTPS settings
6. Execute migrations safely and know how to roll back code vs schema.
7. Validate deployment using:
   - `manage.py check --deploy`
   - `/healthz` and `/readyz`
8. Understand WebSocket proxying requirements when using Channels.

---

## 37.1 Which Deployment Path Should You Choose?

### 37.1.1 Decision table (practical)

**A) VPS (self-managed VM)**
- Good when: you want control, low cost, learning, custom networking
- Tools: Nginx + systemd + Gunicorn/Uvicorn + Postgres/Redis (managed or on VM)
- Pros: maximum control, industry-relevant fundamentals
- Cons: you maintain OS updates, firewall, backups, TLS, process mgmt

**B) PaaS (platform-as-a-service)**
- Good when: you want fast deploys, managed TLS, easy scaling, minimal ops
- Examples: “Heroku-like” or “Render-like” platforms
- Pros: fastest path to production, less OS work
- Cons: platform-specific limits; sometimes less flexibility; costs can grow

**C) Containers (Docker)**
- Good when: you want reproducible builds, consistent envs, Kubernetes later
- Tools: Dockerfile + docker compose (dev), then deploy container to a host/platform
- Pros: reproducibility, best for teams/CI; matches modern industry
- Cons: more moving parts; you still need orchestration/hosting

### 37.1.2 For this workbook
If you want the best “industry fundamentals”:
- do **VPS + Nginx + ASGI** (if you use Channels) or **WSGI** (if not)

If you want the fastest “ship it”:
- do **PaaS** (but keep your config portable)

If you want the best reproducibility:
- do **Docker** (plus a hosting method)

---

## 37.2 WSGI vs ASGI Deployment Choice (Non‑Optional Clarity)

### Choose WSGI (Gunicorn) when:
- you’re mostly server-rendered HTML + normal APIs
- you are not using WebSockets/Channels
- you want simplest deployment

Run:
- `gunicorn config.wsgi:application`

### Choose ASGI (Uvicorn or Gunicorn+UvicornWorker) when:
- you use Channels/WebSockets
- you want async views support without adaptation overhead
- you need long-lived connections or streaming

Run (common):
- `gunicorn config.asgi:application -k uvicorn.workers.UvicornWorker`

**If you use Channels**: you must run ASGI for WebSockets.

---

## 37.3 Deployment Architecture (What Production Usually Looks Like)

### 37.3.1 “Classic” architecture (recommended baseline)
```text
Internet
  |
  | HTTPS
  v
Nginx (TLS termination, gzip, static, proxy)
  |
  | HTTP (internal)
  v
Gunicorn/Uvicorn (Django)
  |
  +--> PostgreSQL
  +--> Redis (cache / Channels / Celery broker)
  +--> Object storage (S3) for media (optional)
```

### 37.3.2 Why Nginx is used
- terminates TLS (HTTPS)
- serves static files efficiently
- handles WebSocket proxying (upgrade headers)
- protects Django from direct internet exposure
- can rate limit and buffer responses

---

## 37.4 Pre‑Deployment Checklist (Don’t Skip This)

Before touching a server:

### 37.4.1 Verify production settings module
- `DEBUG=False`
- `SECRET_KEY` from env
- `ALLOWED_HOSTS` correct
- `CSRF_TRUSTED_ORIGINS` correct for your HTTPS origins
- secure cookies enabled in prod

### 37.4.2 Verify health endpoints exist and behave
- `/healthz` returns 200 always if process is alive
- `/readyz` returns:
  - 200 when DB is OK
  - 503 when DB is down

### 37.4.3 Verify static and media plan
- static: `collectstatic` output served by Nginx/CDN
- media: persistent storage (S3 or disk volume)
- do **not** rely on Django serving static/media in production

### 37.4.4 Verify background and realtime dependencies
If you use Celery:
- broker reachable (Redis)
- worker + beat run as services

If you use Channels:
- ASGI server used
- Redis channel layer configured
- origin validation configured

---

# 37.5 Path 1 — VPS Deployment (Nginx + systemd) (Industry Fundamentals)

This section is a complete “VPS runbook.”

## 37.5.1 Provision the server
- Ubuntu LTS or Debian stable is common
- Ensure:
  - firewall configured (only 22/80/443 open)
  - automatic security updates enabled (recommended)
  - SSH keys used (disable password SSH)

Install system packages:

```bash
sudo apt update
sudo apt install -y \
  python3-venv python3-pip \
  nginx \
  postgresql-client \
  redis-tools \
  git
```

If you run Postgres/Redis on the same VM (not recommended for serious prod), install
those too. Prefer managed Postgres/Redis if possible.

## 37.5.2 Create a deploy user and directory
```bash
sudo adduser --disabled-password django
sudo mkdir -p /srv/myapp
sudo chown django:django /srv/myapp
```

## 37.5.3 Clone your repo
```bash
sudo -u django git clone <your_repo_url> /srv/myapp
```

## 37.5.4 Create venv and install deps
```bash
sudo -u django bash -lc '
  cd /srv/myapp &&
  python3 -m venv .venv &&
  . .venv/bin/activate &&
  python -m pip install --upgrade pip &&
  python -m pip install -r requirements.txt
'
```

If you split dev deps, don’t install dev-only requirements in production.

---

## 37.5.5 Environment variables (systemd-friendly)

Create `/etc/myapp/myapp.env`:

```bash
sudo mkdir -p /etc/myapp
sudo nano /etc/myapp/myapp.env
```

Example content (adjust to your stack):

```text
DJANGO_SETTINGS_MODULE=config.settings.prod
DJANGO_SECRET_KEY=REPLACE_ME
DJANGO_DEBUG=false
DJANGO_ALLOWED_HOSTS=example.com,www.example.com
DJANGO_CSRF_TRUSTED_ORIGINS=https://example.com,https://www.example.com

SITE_URL=https://example.com
SITE_NAME=Django Mastery

POSTGRES_DB=mydb
POSTGRES_USER=myuser
POSTGRES_PASSWORD=REPLACE_ME
POSTGRES_HOST=127.0.0.1
POSTGRES_PORT=5432

CELERY_BROKER_URL=redis://127.0.0.1:6379/0
CELERY_RESULT_BACKEND=redis://127.0.0.1:6379/1

# If using Channels:
REDIS_URL=redis://127.0.0.1:6379/2

DEFAULT_FROM_EMAIL=no-reply@example.com
EMAIL_HOST=smtp.example.com
EMAIL_PORT=587
EMAIL_HOST_USER=...
EMAIL_HOST_PASSWORD=...
EMAIL_USE_TLS=true

WEBHOOK_PROVIDER_X_SECRET=REPLACE_ME
```

Permissions:
```bash
sudo chown root:root /etc/myapp/myapp.env
sudo chmod 600 /etc/myapp/myapp.env
```

### Why systemd env files are standard
- keeps secrets off the filesystem inside repo
- easy to rotate without code changes
- consistent for web/worker/beat services

---

## 37.5.6 Migrations and collectstatic (one-time and per-release)

Run:

```bash
sudo -u django bash -lc '
  cd /srv/myapp &&
  . .venv/bin/activate &&
  python manage.py check --deploy &&
  python manage.py migrate &&
  python manage.py collectstatic --noinput
'
```

### Explain `check --deploy`
It flags common production misconfigurations:
- DEBUG on
- insecure settings
- missing ALLOWED_HOSTS
It’s not perfect, but it’s a good gate.

---

## 37.5.7 systemd service: Django web (WSGI or ASGI)

### Option A: WSGI (Gunicorn)
Install gunicorn if not in requirements:

```bash
sudo -u django bash -lc '
  cd /srv/myapp &&
  . .venv/bin/activate &&
  python -m pip install gunicorn
'
```

Create `/etc/systemd/system/myapp-web.service`:

```ini
[Unit]
Description=MyApp Django Web (Gunicorn WSGI)
After=network.target

[Service]
User=django
Group=django
WorkingDirectory=/srv/myapp
EnvironmentFile=/etc/myapp/myapp.env
ExecStart=/srv/myapp/.venv/bin/gunicorn \
  config.wsgi:application \
  --bind 127.0.0.1:8001 \
  --workers 3 \
  --timeout 60

Restart=always
RestartSec=5

# Security hardening (optional baseline)
NoNewPrivileges=true
PrivateTmp=true
ProtectSystem=strict
ProtectHome=true
ReadWritePaths=/srv/myapp /srv/myapp/media /srv/myapp/staticfiles

[Install]
WantedBy=multi-user.target
```

### Option B: ASGI (Gunicorn + UvicornWorker) (required for Channels/WebSockets)
Install:

```bash
sudo -u django bash -lc '
  cd /srv/myapp &&
  . .venv/bin/activate &&
  python -m pip install gunicorn uvicorn
'
```

Create `/etc/systemd/system/myapp-web.service`:

```ini
[Unit]
Description=MyApp Django Web (Gunicorn ASGI via UvicornWorker)
After=network.target

[Service]
User=django
Group=django
WorkingDirectory=/srv/myapp
EnvironmentFile=/etc/myapp/myapp.env
ExecStart=/srv/myapp/.venv/bin/gunicorn \
  config.asgi:application \
  -k uvicorn.workers.UvicornWorker \
  --bind 127.0.0.1:8001 \
  --workers 2 \
  --timeout 60

Restart=always
RestartSec=5

NoNewPrivileges=true
PrivateTmp=true
ProtectSystem=strict
ProtectHome=true
ReadWritePaths=/srv/myapp /srv/myapp/media /srv/myapp/staticfiles

[Install]
WantedBy=multi-user.target
```

### Why bind to 127.0.0.1:8001
- Django app is not exposed directly to the internet
- Nginx proxies to it locally
- reduces attack surface

Enable and start:

```bash
sudo systemctl daemon-reload
sudo systemctl enable --now myapp-web
sudo systemctl status myapp-web --no-pager
```

Logs:
```bash
sudo journalctl -u myapp-web -f
```

---

## 37.5.8 systemd services: Celery worker and beat (if you use Celery)

### Celery worker
Create `/etc/systemd/system/myapp-celery-worker.service`:

```ini
[Unit]
Description=MyApp Celery Worker
After=network.target

[Service]
User=django
Group=django
WorkingDirectory=/srv/myapp
EnvironmentFile=/etc/myapp/myapp.env
ExecStart=/srv/myapp/.venv/bin/celery \
  -A config worker \
  -l INFO

Restart=always
RestartSec=5

NoNewPrivileges=true
PrivateTmp=true
ProtectSystem=strict
ProtectHome=true
ReadWritePaths=/srv/myapp /srv/myapp/media

[Install]
WantedBy=multi-user.target
```

### Celery beat (scheduler)
Create `/etc/systemd/system/myapp-celery-beat.service`:

```ini
[Unit]
Description=MyApp Celery Beat
After=network.target

[Service]
User=django
Group=django
WorkingDirectory=/srv/myapp
EnvironmentFile=/etc/myapp/myapp.env
ExecStart=/srv/myapp/.venv/bin/celery \
  -A config beat \
  -l INFO

Restart=always
RestartSec=5

NoNewPrivileges=true
PrivateTmp=true
ProtectSystem=strict
ProtectHome=true
ReadWritePaths=/srv/myapp /srv/myapp/media

[Install]
WantedBy=multi-user.target
```

Enable:

```bash
sudo systemctl daemon-reload
sudo systemctl enable --now myapp-celery-worker
sudo systemctl enable --now myapp-celery-beat
```

---

## 37.5.9 Nginx configuration (HTTP + static + proxy + WebSockets)

Create `/etc/nginx/sites-available/myapp`:

```nginx
server {
  listen 80;
  server_name example.com www.example.com;

  # Health checks can be allowed without auth.
  location /healthz/ {
    proxy_pass http://127.0.0.1:8001;
    proxy_set_header Host $host;
    proxy_set_header X-Forwarded-Proto $scheme;
  }

  location /readyz/ {
    proxy_pass http://127.0.0.1:8001;
    proxy_set_header Host $host;
    proxy_set_header X-Forwarded-Proto $scheme;
  }

  # Static files (collected)
  location /static/ {
    alias /srv/myapp/staticfiles/;
    access_log off;
    expires 30d;
    add_header Cache-Control "public, max-age=2592000";
  }

  # Media files (public uploads) - only if you decided media is public
  # If media is private, do NOT serve it here; use authenticated views or signed URLs.
  location /media/ {
    alias /srv/myapp/media/;
    access_log off;
    expires 7d;
  }

  # WebSocket endpoint (Channels)
  # Only needed if you use WebSockets under /ws/
  location /ws/ {
    proxy_pass http://127.0.0.1:8001;
    proxy_http_version 1.1;
    proxy_set_header Upgrade $http_upgrade;
    proxy_set_header Connection "upgrade";
    proxy_set_header Host $host;
    proxy_set_header X-Forwarded-Proto $scheme;
  }

  # Default proxy to Django
  location / {
    proxy_pass http://127.0.0.1:8001;
    proxy_set_header Host $host;
    proxy_set_header X-Forwarded-Proto $scheme;
    proxy_set_header X-Request-Id $request_id;
    proxy_redirect off;
  }
}
```

Enable it:

```bash
sudo ln -s /etc/nginx/sites-available/myapp /etc/nginx/sites-enabled/myapp
sudo nginx -t
sudo systemctl reload nginx
```

### Why separate /ws/ location is needed
WebSockets require:
- HTTP/1.1
- Upgrade headers
If you proxy `/ws/` like normal HTTP without upgrade headers, sockets won’t work.

---

## 37.5.10 HTTPS (Let’s Encrypt) (VPS standard)
Install certbot:

```bash
sudo apt install -y certbot python3-certbot-nginx
```

Obtain certificate:

```bash
sudo certbot --nginx -d example.com -d www.example.com
```

Certbot updates Nginx config to include TLS and sets up renewal.

### Post‑TLS checks
- Ensure Django knows original scheme if behind proxy:
  - set `SECURE_PROXY_SSL_HEADER` if your proxy sets `X-Forwarded-Proto`
- Ensure `CSRF_TRUSTED_ORIGINS` includes `https://example.com`

---

## 37.5.11 Validate the deployment (must-do smoke tests)

From your laptop:
- Visit `https://example.com/`
- Visit `https://example.com/admin/` (CSS loads)
- `curl -i https://example.com/healthz/` (200)
- `curl -i https://example.com/readyz/` (200)
- Create/edit content and ensure:
  - static assets load
  - media upload works (if enabled)
  - background tasks execute (Celery logs show)
  - WebSockets connect (if enabled)

---

# 37.6 Path 2 — PaaS Deployment (Render/Heroku-like) (Portable Guidance)

PaaS details vary by provider, but the industry pattern is consistent:

## 37.6.1 What PaaS usually provides
- build system (pip install)
- runtime process management
- environment variables/secrets UI
- TLS and domains (often managed)
- add-ons for PostgreSQL/Redis (or external managed services)
- logs aggregation

## 37.6.2 What you must provide
- a start command
- settings via env vars
- database migrations step
- collectstatic step (or a static files strategy)
- separate worker process for Celery

### Typical process commands
**Web (WSGI):**
```bash
gunicorn config.wsgi:application --bind 0.0.0.0:$PORT --workers 3
```

**Web (ASGI):**
```bash
gunicorn config.asgi:application \
  -k uvicorn.workers.UvicornWorker \
  --bind 0.0.0.0:$PORT \
  --workers 2
```

**Worker:**
```bash
celery -A config worker -l info
```

**Beat:**
```bash
celery -A config beat -l info
```

## 37.6.3 Static files on PaaS
Three common strategies:

1) Platform supports serving static from `STATIC_ROOT` after collectstatic  
2) Use a library like WhiteNoise to serve static from Django (acceptable for many
   medium apps; less ideal for heavy static traffic)  
3) Use object storage/CDN for static assets

WhiteNoise is common because it simplifies:
- no Nginx needed
- immutable caching headers
But it’s still “Django serves static,” so understand tradeoffs.

## 37.6.4 Migrations on PaaS
Industry standard:
- run migrations as a release step
- ensure only one migration runner at a time (avoid race)
- use expand/contract pattern for risky migrations

---

# 37.7 Path 3 — Docker Deployment (Reproducible Builds)

This is the “modern baseline” many teams prefer because:
- CI can build the same container you deploy
- dependencies are isolated and reproducible
- scaling to Kubernetes later is straightforward

## 37.7.1 Minimal Dockerfile (production-style)
Create `Dockerfile`:

```dockerfile
FROM python:3.12-slim

ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

WORKDIR /app

# System deps (psycopg and build tools if needed)
RUN apt-get update && apt-get install -y --no-install-recommends \
  build-essential \
  && rm -rf /var/lib/apt/lists/*

COPY requirements.txt /app/
RUN python -m pip install --upgrade pip && \
  python -m pip install -r requirements.txt

COPY . /app/

# Collect static at build time OR at release time.
# Many teams do it at release time so env-specific storages work.
# RUN python manage.py collectstatic --noinput

EXPOSE 8000

CMD ["gunicorn", "config.asgi:application", "-k", "uvicorn.workers.UvicornWorker", "--bind", "0.0.0.0:8000", "--workers", "2", "--timeout", "60"]
```

### Explain key choices
- `PYTHONUNBUFFERED=1`: logs appear immediately
- install requirements before copying source: better Docker layer caching
- ASGI command by default (works for both HTTP and WebSockets)

## 37.7.2 Docker Compose for local “prod-like” stack
Create `docker-compose.prod.yml`:

```yaml
services:
  web:
    build: .
    env_file:
      - .env.prod
    ports:
      - "8000:8000"
    depends_on:
      - db
      - redis

  worker:
    build: .
    env_file:
      - .env.prod
    command: celery -A config worker -l INFO
    depends_on:
      - db
      - redis

  beat:
    build: .
    env_file:
      - .env.prod
    command: celery -A config beat -l INFO
    depends_on:
      - db
      - redis

  db:
    image: postgres:16-alpine
    environment:
      POSTGRES_DB: django_app
      POSTGRES_USER: django
      POSTGRES_PASSWORD: django
    volumes:
      - pgdata:/var/lib/postgresql/data

  redis:
    image: redis:7-alpine
    ports:
      - "6379:6379"

volumes:
  pgdata:
```

Create `.env.prod` (do not commit real secrets):

```text
DJANGO_SETTINGS_MODULE=config.settings.prod
DJANGO_SECRET_KEY=change-me
DJANGO_DEBUG=false
DJANGO_ALLOWED_HOSTS=localhost,127.0.0.1
DJANGO_CSRF_TRUSTED_ORIGINS=http://localhost:8000

POSTGRES_DB=django_app
POSTGRES_USER=django
POSTGRES_PASSWORD=django
POSTGRES_HOST=db
POSTGRES_PORT=5432

CELERY_BROKER_URL=redis://redis:6379/0
CELERY_RESULT_BACKEND=redis://redis:6379/1
```

Run:

```bash
docker compose -f docker-compose.prod.yml up --build
```

Then run migrations inside the web container:

```bash
docker compose -f docker-compose.prod.yml exec web python manage.py migrate
docker compose -f docker-compose.prod.yml exec web python manage.py collectstatic --noinput
```

### Production note
In a real production container environment:
- migrations and collectstatic are often done in a “release job” step, not on a
  running web container
- media should be stored in object storage or a persistent volume
- static should be served via CDN or reverse proxy

---

# 37.8 WebSockets in Production (Channels-Specific Requirements)

If you use Channels:

1) You must run an ASGI server (Uvicorn/Daphne/Gunicorn+UvicornWorker).  
2) You must proxy WebSockets correctly at the edge (Nginx upgrade headers).  
3) You should use Redis channel layer (not in-memory).  
4) You must validate origins (`AllowedHostsOriginValidator` or explicit
   `OriginValidator`).  
5) Set correct timeouts at proxy and ASGI server for long-lived connections.

A very common mistake is:
- deploying with Gunicorn WSGI and expecting WebSockets to work. They won’t.

---

# 37.9 Migrations and Release Flow (Professional Minimal Downtime)

A practical release flow used in many teams:

1. Build artifact (wheel/container)
2. Put app into “drain” mode if needed (optional)
3. Run DB migrations (safe expand/contract discipline)
4. Run collectstatic (if needed)
5. Restart web workers
6. Restart celery workers (optional, depends on task code compatibility)
7. Run smoke tests (`/readyz`, key pages)
8. Monitor logs and error rate

### Rollback reality
- Code rollback is usually easy.
- Schema rollback is often hard or unsafe.
That’s why expand/contract is so important.

---

# 37.10 Hardening and Verification Commands (Use These Before Declaring Success)

Run on production build (or staging):

```bash
python manage.py check --deploy
python manage.py migrate --check
python manage.py showmigrations --plan
```

Then confirm:
- `/healthz` 200
- `/readyz` 200
- admin static works
- celery worker logs show tasks execute
- websocket connects (if applicable)

---

## 37.11 Chapter Capstone Lab (Pick ONE and Complete It)

### Lab A (best fundamentals): VPS + Nginx + systemd
- Deploy web service
- Deploy worker and beat
- Configure Nginx (static + proxy + ws)
- Add TLS
- Validate health endpoints

### Lab B (fastest to ship): PaaS
- Configure env vars
- Configure start commands
- Add worker process
- Configure migrations step
- Validate via logs + health endpoints

### Lab C (most reproducible): Docker on a server
- Build container
- Run with compose
- Externalize Postgres/Redis if possible
- Add reverse proxy for TLS
- Validate behavior

---

## 37.12 Exercises (Do These Before Proceeding)

1. Write a `docs/deploy.md` runbook that includes:
   - preflight checks
   - migration steps
   - collectstatic steps
   - restart steps
   - smoke tests
2. Add a “staging” environment config and deploy there first.
3. Add a “release check” script/command that verifies:
   - DB connectivity (`/readyz`)
   - migrations applied
   - celery broker reachable (optional)
4. If you use Channels, add a WebSocket smoke test (manual or automated).

---

## 37.13 Chapter Summary
- Production Django is multiple processes (web + workers) behind a reverse proxy.
- Choose WSGI for classic apps; choose ASGI if you need WebSockets/async at scale.
- Static is served by Nginx/CDN; media must be persistent (disk volume or S3).
- Migrations require discipline (expand/contract) because schema rollbacks are hard.
- Health endpoints + logs + request IDs are your operational foundation.

---

Next chapter: **38. CI/CD (Automation that Professionals Expect)**  
We’ll set up a pipeline that runs lint/format/tests/migrations checks, builds an
artifact/container, deploys safely, and supports rollbacks—matching industry
expectations.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='36. production_readiness_checklist.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='38. ci_cd.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
